0=1索　1=赤1索　2=2索　3=赤2索　4=3索　5=赤3索　…　14=8索　15=赤8索　16=9索　17=赤9索　18=發　19=中

**meanings of variables**

- hand : hand tiles (ex hand = [0, 0, 1, 2, 2, 2], len(hand) == 6)

- discard_hand : hand tiles when discard one tile (ex hand = [0, 1, 2, 2, 2], len(discard_hand) == 5)

- NUM_SAME_TILE : number of same tiles（4）

- KIND_TILE : number of kinds of tiles（1~9索 + 發 + 中 = 11）

- MAX_LEN_HAND : length of hand tiles（6）


In [13]:
import itertools
import numpy as np
import pickle

NUM_SAME_TILE = 4
KIND_TILE = 11
KIND_TILE_WITH_RED = 20
MAX_LEN_HAND = 6

In [2]:
# "len_hand" is MAX_LEN_HAND or MAX_LEN_HAND - 1
def generate_all_valid_hands(len_hand, dora=None):
    all_valid_hands = []
    for hand in itertools.combinations_with_replacement(range(KIND_TILE_WITH_RED), len_hand):
        if (is_valid(hand, dora)):
            all_valid_hands.append(hand)
    return all_valid_hands

# check each kind of tile
def is_valid(hand, dora=None):
    hand_set = set(hand)
    for tile in hand_set:
        if tile == dora:
            dora_count = 1
        else:
            dora_count = 0
            
        if tile == 18 or tile == 19: # 發 or 中
            if hand.count(tile) >= NUM_SAME_TILE + 1 - dora_count:
                return False
        elif tile % 2 == 0: # normal tile (not red Dora tile)
            if hand.count(tile) >= 4 - dora_count:
                return False
        else: # red Dora tile
            if hand.count(tile) >= 2 - dora_count:
                return False
    return True

def hand2show_str(hand):
    show_str = ''
    for tile in hand:
        if tile == 18:
            show_str += '發 '
            continue
        elif tile == 19:
            show_str += '中 '
            continue
        elif tile % 2 == 1:
            tile -= 1
            show_str += 'r'
        num = int((tile / 2) + 1)
        show_str += '{} '.format(num)
    return show_str

# valid_hands = generate_all_valid_hands(MAX_LEN_HAND)
# valid_discard_hands = generate_all_valid_hands(MAX_LEN_HAND - 1)
#print(valid_hands[20:50])
# print(valid_discard_hands[-50:])
# print(len(valid_hands))
# print(len(valid_discard_hands))

In [3]:
def hand2hist(hand):
    # hist = [1索, 2索, ..., 9索, 0, 發, 0, 中]
    # hist[9] and hist[11] are always 0 due to has_2melds function
    hist = [0] * (KIND_TILE + 2)
    for tile in hand: 
        if tile == 19:
            hist[12] += 1
        elif tile == 18:
            hist[10] += 1
        else:
            hist[tile // 2] += 1
    return hist

def hands2hists(hands):
    hists = []
    for hand in hands:
        hists.append(hand2hist(hand))
    return hists

def has_2melds(hist):
    return has_2melds_sub(hist, n_melds=2)

def has_2melds_sub(hist, n_melds):
    if n_melds == 0:
        return True
    i = next(i for i, x in enumerate(hist) if x > 0)
    # Pong
    if hist[i] >= 3 and has_2melds_sub([x - 3 if i == j else x for j, x in enumerate(hist)], n_melds - 1):
        return True
    # Chows
    if i + 2 < len(hist) and hist[i + 1] > 0 and hist[i + 2] > 0 and has_2melds_sub([x - 1 if i <= j <= i + 2 else x for j, x in enumerate(hist)], n_melds - 1):
        return True
    return False

In [4]:
# yaku
one_nine_honer_tiles = [0, 1, 16, 17, 18, 19] # 1索, 9索, 發, 中

# count odd number
def count_red_tile(hand):
    count = 0
    for tile in hand:
        if tile % 2 == 1:
            count += 1
    return count

# include one_nine_honer_tile or not
def is_tanyao(hand):
    for tile in hand:
        if tile in one_nine_honer_tiles:
            return False
    return True

# count 1索, 9索, 發, 中
# chanta <=> has_2melds(hand) == True and (count_one_nine_honer == 2 or count_one_nine_honer == 4)
def is_chanta(hand):
    count_one_nine_honor = 0
    for tile in hand:
        if tile in one_nine_honer_tiles:
            count_one_nine_honor += 1
    if count_one_nine_honor == 2 or count_one_nine_honor == 4:
        return True
    return False

# include green tile or not
def is_all_green(hand):
    green_tiles = [2, 4, 6, 10, 14, 18] # 2索, 3索, 4索, 6索, 8索, 發
    for tile in hand:
        if not tile in green_tiles:
            return False
    return True

# count 1索, 9索, 發, 中
def is_chinyao(hand):
    count_one_nine_honor = 0
    for tile in hand:
        if tile in one_nine_honer_tiles:
            count_one_nine_honor += 1
    if count_one_nine_honor == 6:
        return True
    return False

# count red tile
def is_super_red(hand):
    for tile in hand:
        if tile % 2 == 0: # even number is not red
            return False
    return True

def calc_point(hand, dora=None):
    hist = hand2hist(hand)
    if not has_2melds(hist):
        return 0
    else:
        if is_all_green(hand):
            return 10
        if is_chinyao(hand):
            return 15
        if is_super_red(hand):
            return 20
        
        point = count_red_tile(hand)
        if hist.count(3) == 2:
            point += 4 # 2 Pongs
        elif hist.count(3) == 1 or hist.count(4) == 1:
            point += 3 # 1 Pong and 1 Chow
        else:
            point += 2 # 2 Chows
        if is_tanyao(hand):
            point += 1
        elif is_chanta(hand):
            point += 2
        if dora is not None:
            if dora == 18 or dora == 19:
                point += hand.count(dora)
            else:
                if dora % 2 == 1:
                    point += hand.count(dora - 1)
                else:
                    point += hand.count(dora)
                    point += hand.count(dora + 1)
        return point

In [5]:
# test
import unittest

class TestMethod(unittest.TestCase):
    def test_has_2melds(self):
        self.assertTrue(has_2melds(hand2hist([0, 0, 2, 2, 4, 4])))
        self.assertTrue(has_2melds(hand2hist([0, 0, 1, 19, 19, 19])))
        self.assertTrue(has_2melds(hand2hist([0, 2, 5, 6, 8, 10])))
        self.assertTrue(has_2melds(hand2hist([0, 0, 2, 2, 4, 4])))
        self.assertFalse(has_2melds(hand2hist([0, 1, 2, 10, 10, 10])))
        self.assertFalse(has_2melds(hand2hist([0, 0, 0, 17, 18, 19])))
        self.assertFalse(has_2melds(hand2hist([0, 0, 1, 16, 17, 18])))
        self.assertTrue(has_2melds(hand2hist([0, 0, 1, 16, 16, 17])))
        
    def test_calc_point(self):
        self.assertEqual(calc_point([1, 3, 5, 19, 19, 19]), 20)
        self.assertEqual(calc_point([3, 5, 7, 15, 17, 19]), 0)
        self.assertEqual(calc_point([18, 18, 18, 19, 19, 19]), 15)
        self.assertEqual(calc_point([18, 18, 18, 18, 19, 19]), 0)
        self.assertEqual(calc_point([2, 2, 4, 4, 6, 6]), 10)
        self.assertEqual(calc_point([10, 10, 10, 18, 18, 18]), 10)
        self.assertEqual(calc_point([3, 4, 6, 10, 12, 14]), 4)
        self.assertEqual(calc_point([0, 0, 2, 2, 4, 4]), 4)
        self.assertEqual(calc_point([4, 4, 4, 19, 19, 19]), 7)
        self.assertEqual(calc_point([10, 10, 11, 18, 18, 18]), 5)
        self.assertEqual(calc_point([0, 2, 4, 10, 12, 14]), 2)
        self.assertEqual(calc_point([0, 2, 5, 12, 14, 17]), 6)
        self.assertEqual(calc_point([0, 2, 5, 12, 14, 17], dora=1), 7)
        self.assertEqual(calc_point([0, 2, 5, 12, 14, 17], dora=0), 7)
        self.assertEqual(calc_point([0, 2, 5, 12, 14, 17], dora=11), 6)
        self.assertEqual(calc_point([10, 10, 11, 18, 18, 18], dora=18), 8)
        self.assertEqual(calc_point([18, 18, 18, 19, 19, 19], dora=19), 15)

suite = unittest.TestSuite()
suite.addTest(TestMethod('test_has_2melds'))
suite.addTest(TestMethod('test_calc_point'))
runner = unittest.TextTestRunner()
runner.run(suite)

..
----------------------------------------------------------------------
Ran 2 tests in 0.003s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

In [6]:
# wall = [0, 0, 0, 1, 2, 2, 2, 3, 4, 4, 4, 5, ..., 16, 16, 16, 17, 18, 18, 18, 18, 19, 19, 19, 19]
# if set 'dora', remove dora from wall
def make_wall(dora=None):
    wall = list(range(KIND_TILE_WITH_RED))
    for i in range(0, 18, 2):
        for j in range(2):
            wall.append(i)
    for i in [18, 19]:
        for j in range(3):
            wall.append(i)
    if dora is not None: # remove dora from wall
        wall.remove(dora)
    return sorted(wall)

# FIXME: if discard_hand includes dora, wall.remove raise Error
def discard_hand_to_hand_idx_and_prob(discard_hand, valid_hands, dora=None):
    wall = make_wall(dora)
    for tile in discard_hand:
        wall.remove(tile)
    hand_idx_and_prob = []
    for tile in range(KIND_TILE_WITH_RED):
        if wall.count(tile) == 0:
            continue
        prob = wall.count(tile) / len(wall)
        next_hand = tuple(sorted(list(discard_hand) + [tile]))
        hand_idx = valid_hands.index(next_hand)
        hand_idx_and_prob.append((hand_idx, prob))
    return hand_idx_and_prob

def hand_to_discard_hands(hand):
    return list(set(tuple(hand[:i] + hand[i+1:]) for i in range(MAX_LEN_HAND)))

# it takes about 15 minites
def value_iteration(gamma, dora=None):
    valid_hands = generate_all_valid_hands(MAX_LEN_HAND, dora)
    valid_discard_hands = generate_all_valid_hands(MAX_LEN_HAND - 1, dora)
    hand_value = []
    for hand in valid_hands:
        point = calc_point(hand, dora)
        if point <= 4:
            hand_value.append(0)
        else:
            hand_value.append(point)
    discard_hand_value = [0] * len(valid_discard_hands)
    hand_idx_and_prob = [discard_hand_to_hand_idx_and_prob(discard_hand, valid_hands, dora) for discard_hand in valid_discard_hands]
    hand_to_discard_hand_idx = [[valid_discard_hands.index(discard_hand) for discard_hand in hand_to_discard_hands(hand)] for hand in valid_hands]
    theta = 1e-6
    while True:
#         print('iteration')
        delta = 0
        for idx, discard_hand in enumerate(valid_discard_hands):
            old_value = discard_hand_value[idx]
            discard_hand_value[idx] = sum(p * hand_value[i] for (i, p) in hand_idx_and_prob[idx])
            delta = max(delta, abs(old_value - discard_hand_value[idx]))
        if delta < theta:
            break
        for idx, hand in enumerate(valid_hands):
            if has_2melds(hand2hist(hand)):
                continue
            hand_value[idx] = max(gamma * discard_hand_value[i] for i in hand_to_discard_hand_idx[idx])
    return discard_hand_value

In [7]:
# value iteration and save values
# for tile in range(KIND_TILE_WITH_RED):
# for tile in [None]:
#     print(tile)
#     vi = value_iteration(0.6, dora=tile)
#     np.save('val_ite_06_{}'.format(tile), np.array(vi))

# save hand_idx_and_prob
# for dora in [None]:
#     print(dora)
#     valid_hands = generate_all_valid_hands(MAX_LEN_HAND, dora)
#     #print(valid_hands[:10])
#     valid_discard_hands = generate_all_valid_hands(MAX_LEN_HAND - 1, dora)
#     #print(discard_valid_hands[:10])
#     hand_idx_and_prob = [discard_hand_to_hand_idx_and_prob(discard_hand, valid_hands, dora) for discard_hand in valid_discard_hands]
#     np.save('hand_idx_and_prob_{}'.format(dora), np.array(hand_idx_and_prob))

In [8]:
# show results (point)
dora = None
valid_discard_hands = generate_all_valid_hands(MAX_LEN_HAND - 1, dora)
discard_hand_value = np.load('value_iteration/val_ite_09_{}.npy'.format(dora)).tolist()
for i, discard_hand in enumerate(valid_discard_hands):
    if discard_hand_value[i] > 7:
        print('discard_hand {}\t{}\t value {}'.format(discard_hand, hand2show_str(discard_hand), discard_hand_value[i]))

discard_hand (1, 3, 5, 9, 11)	r1 r2 r3 r5 r6 	 value 7.387387387386788
discard_hand (1, 3, 5, 11, 13)	r1 r2 r3 r6 r7 	 value 7.427927927927289
discard_hand (1, 3, 5, 13, 15)	r1 r2 r3 r7 r8 	 value 7.927927927927288
discard_hand (1, 3, 5, 19, 19)	r1 r2 r3 中 中 	 value 7.017539178510319
discard_hand (3, 5, 7, 9, 11)	r2 r3 r4 r5 r6 	 value 7.898550724637679
discard_hand (3, 5, 7, 11, 13)	r2 r3 r4 r6 r7 	 value 7.927927927927288
discard_hand (3, 5, 7, 13, 15)	r2 r3 r4 r7 r8 	 value 7.677927927927287
discard_hand (3, 5, 7, 19, 19)	r2 r3 r4 中 中 	 value 7.017539178510319
discard_hand (3, 5, 9, 11, 13)	r2 r3 r5 r6 r7 	 value 7.659177927927288
discard_hand (3, 5, 11, 13, 15)	r2 r3 r6 r7 r8 	 value 7.677927927927287
discard_hand (3, 5, 13, 15, 17)	r2 r3 r7 r8 r9 	 value 7.927927927927288
discard_hand (3, 5, 19, 19, 19)	r2 r3 中 中 中 	 value 8.468468468467782
discard_hand (5, 7, 9, 11, 13)	r3 r4 r5 r6 r7 	 value 8.115942028985504
discard_hand (5, 7, 9, 13, 15)	r3 r4 r5 r7 r8 	 value 7.65917792792729

In [9]:
# 'win hand' is a hand which can get more than 5 points
def has_win_hand(hand_idx_and_prob_i, valid_hands):
    for (idx, p) in hand_idx_and_prob_i:
        if calc_point(valid_hands[idx]) >= 5:
            return True
    return False

dora = None
valid_hands = generate_all_valid_hands(MAX_LEN_HAND, dora)
valid_discard_hands = generate_all_valid_hands(MAX_LEN_HAND - 1, dora)
discard_hand_value = np.load('value_iteration/val_ite_06_{}.npy'.format(dora)).tolist()
hand_idx_and_prob = np.load('hand_idx_prob/hand_idx_and_prob_{}.npy'.format(dora)).tolist()

# smallest discard hand value
(min_value, min_discard_hand) = min((discard_hand_value[idx], discard_hand) for idx, discard_hand in enumerate(valid_discard_hands))
print('{} {} {}'.format(min_value, min_discard_hand, hand2show_str(min_discard_hand)))

# largest discard hand value
(max_value, max_discard_hand) = max((discard_hand_value[idx], discard_hand) for idx, discard_hand in enumerate(valid_discard_hands))
print('{} {} {}'.format(max_value, max_discard_hand, hand2show_str(max_discard_hand)))

0.07639890139682579 (0, 10, 16, 18, 19) 1 6 9 發 中 
5.045045043484679 (5, 7, 9, 11, 13) r3 r4 r5 r6 r7 


In [10]:
# 'ready' means that a discard hand has at least one win hand
is_ready = [has_win_hand(hand_idx_and_prob[i], valid_hands) for i in range(len(valid_discard_hands))]
# if 'ready' means that a discard hand has at least one hand which has 2 melds, use next 'is_ready'
# is_ready = [any(has_2melds(hand2hist(valid_hands[idx])) for (idx, p) in hand_idx_and_prob[i]) for i, discard_hand in enumerate(valid_discard_hands)]

# smallest discard hand value which is ready
(min_value, min_discard_hand) = min((discard_hand_value[idx], discard_hand) for idx, discard_hand in enumerate(valid_discard_hands) if is_ready[idx])
print('{} {} {}'.format(min_value, min_discard_hand, hand2show_str(min_discard_hand)))

# largest discard hand value which is not ready
(max_value, max_discard_hand) = max((discard_hand_value[idx], discard_hand) for idx, discard_hand in enumerate(valid_discard_hands) if not is_ready[idx])
print('{} {} {}'.format(max_value, max_discard_hand, hand2show_str(max_discard_hand)))

0.3151215628306345 (12, 12, 14, 14, 16) 7 7 8 8 9 
1.5377514919691337 (5, 13, 19, 19, 19) r3 r7 中 中 中 


In [11]:
# show all discard hand
for i, discard_hand in enumerate(valid_discard_hands):
    print(is_ready[i], discard_hand, hand2show_str(discard_hand))

True (0, 0, 0, 1, 2) 1 1 1 r1 2 
True (0, 0, 0, 1, 3) 1 1 1 r1 r2 
True (0, 0, 0, 1, 4) 1 1 1 r1 3 
True (0, 0, 0, 1, 5) 1 1 1 r1 r3 
False (0, 0, 0, 1, 6) 1 1 1 r1 4 
False (0, 0, 0, 1, 7) 1 1 1 r1 r4 
False (0, 0, 0, 1, 8) 1 1 1 r1 5 
False (0, 0, 0, 1, 9) 1 1 1 r1 r5 
False (0, 0, 0, 1, 10) 1 1 1 r1 6 
False (0, 0, 0, 1, 11) 1 1 1 r1 r6 
False (0, 0, 0, 1, 12) 1 1 1 r1 7 
False (0, 0, 0, 1, 13) 1 1 1 r1 r7 
False (0, 0, 0, 1, 14) 1 1 1 r1 8 
False (0, 0, 0, 1, 15) 1 1 1 r1 r8 
False (0, 0, 0, 1, 16) 1 1 1 r1 9 
False (0, 0, 0, 1, 17) 1 1 1 r1 r9 
False (0, 0, 0, 1, 18) 1 1 1 r1 發 
False (0, 0, 0, 1, 19) 1 1 1 r1 中 
True (0, 0, 0, 2, 2) 1 1 1 2 2 
True (0, 0, 0, 2, 3) 1 1 1 2 r2 
True (0, 0, 0, 2, 4) 1 1 1 2 3 
True (0, 0, 0, 2, 5) 1 1 1 2 r3 
False (0, 0, 0, 2, 6) 1 1 1 2 4 
True (0, 0, 0, 2, 7) 1 1 1 2 r4 
False (0, 0, 0, 2, 8) 1 1 1 2 5 
False (0, 0, 0, 2, 9) 1 1 1 2 r5 
False (0, 0, 0, 2, 10) 1 1 1 2 6 
False (0, 0, 0, 2, 11) 1 1 1 2 r6 
False (0, 0, 0, 2, 12) 1 1 1 2 7 
False (0

False (0, 0, 3, 16, 17) 1 1 r2 9 r9 
False (0, 0, 3, 16, 18) 1 1 r2 9 發 
False (0, 0, 3, 16, 19) 1 1 r2 9 中 
False (0, 0, 3, 17, 18) 1 1 r2 r9 發 
False (0, 0, 3, 17, 19) 1 1 r2 r9 中 
False (0, 0, 3, 18, 18) 1 1 r2 發 發 
False (0, 0, 3, 18, 19) 1 1 r2 發 中 
False (0, 0, 3, 19, 19) 1 1 r2 中 中 
True (0, 0, 4, 4, 4) 1 1 3 3 3 
True (0, 0, 4, 4, 5) 1 1 3 3 r3 
False (0, 0, 4, 4, 6) 1 1 3 3 4 
False (0, 0, 4, 4, 7) 1 1 3 3 r4 
False (0, 0, 4, 4, 8) 1 1 3 3 5 
False (0, 0, 4, 4, 9) 1 1 3 3 r5 
False (0, 0, 4, 4, 10) 1 1 3 3 6 
False (0, 0, 4, 4, 11) 1 1 3 3 r6 
False (0, 0, 4, 4, 12) 1 1 3 3 7 
False (0, 0, 4, 4, 13) 1 1 3 3 r7 
False (0, 0, 4, 4, 14) 1 1 3 3 8 
False (0, 0, 4, 4, 15) 1 1 3 3 r8 
False (0, 0, 4, 4, 16) 1 1 3 3 9 
False (0, 0, 4, 4, 17) 1 1 3 3 r9 
False (0, 0, 4, 4, 18) 1 1 3 3 發 
False (0, 0, 4, 4, 19) 1 1 3 3 中 
False (0, 0, 4, 5, 6) 1 1 3 r3 4 
False (0, 0, 4, 5, 7) 1 1 3 r3 r4 
False (0, 0, 4, 5, 8) 1 1 3 r3 5 
False (0, 0, 4, 5, 9) 1 1 3 r3 r5 
False (0, 0, 4, 5, 10) 1 1 3

False (0, 0, 10, 15, 17) 1 1 6 r8 r9 
False (0, 0, 10, 15, 18) 1 1 6 r8 發 
False (0, 0, 10, 15, 19) 1 1 6 r8 中 
False (0, 0, 10, 16, 16) 1 1 6 9 9 
False (0, 0, 10, 16, 17) 1 1 6 9 r9 
False (0, 0, 10, 16, 18) 1 1 6 9 發 
False (0, 0, 10, 16, 19) 1 1 6 9 中 
False (0, 0, 10, 17, 18) 1 1 6 r9 發 
False (0, 0, 10, 17, 19) 1 1 6 r9 中 
False (0, 0, 10, 18, 18) 1 1 6 發 發 
False (0, 0, 10, 18, 19) 1 1 6 發 中 
False (0, 0, 10, 19, 19) 1 1 6 中 中 
False (0, 0, 11, 12, 12) 1 1 r6 7 7 
False (0, 0, 11, 12, 13) 1 1 r6 7 r7 
True (0, 0, 11, 12, 14) 1 1 r6 7 8 
True (0, 0, 11, 12, 15) 1 1 r6 7 r8 
False (0, 0, 11, 12, 16) 1 1 r6 7 9 
False (0, 0, 11, 12, 17) 1 1 r6 7 r9 
False (0, 0, 11, 12, 18) 1 1 r6 7 發 
False (0, 0, 11, 12, 19) 1 1 r6 7 中 
True (0, 0, 11, 13, 14) 1 1 r6 r7 8 
True (0, 0, 11, 13, 15) 1 1 r6 r7 r8 
False (0, 0, 11, 13, 16) 1 1 r6 r7 9 
False (0, 0, 11, 13, 17) 1 1 r6 r7 r9 
False (0, 0, 11, 13, 18) 1 1 r6 r7 發 
False (0, 0, 11, 13, 19) 1 1 r6 r7 中 
False (0, 0, 11, 14, 14) 1 1 r6 8 8 

False (0, 1, 5, 8, 16) 1 r1 r3 5 9 
False (0, 1, 5, 8, 17) 1 r1 r3 5 r9 
False (0, 1, 5, 8, 18) 1 r1 r3 5 發 
False (0, 1, 5, 8, 19) 1 r1 r3 5 中 
False (0, 1, 5, 9, 10) 1 r1 r3 r5 6 
False (0, 1, 5, 9, 11) 1 r1 r3 r5 r6 
False (0, 1, 5, 9, 12) 1 r1 r3 r5 7 
False (0, 1, 5, 9, 13) 1 r1 r3 r5 r7 
False (0, 1, 5, 9, 14) 1 r1 r3 r5 8 
False (0, 1, 5, 9, 15) 1 r1 r3 r5 r8 
False (0, 1, 5, 9, 16) 1 r1 r3 r5 9 
False (0, 1, 5, 9, 17) 1 r1 r3 r5 r9 
False (0, 1, 5, 9, 18) 1 r1 r3 r5 發 
False (0, 1, 5, 9, 19) 1 r1 r3 r5 中 
False (0, 1, 5, 10, 10) 1 r1 r3 6 6 
False (0, 1, 5, 10, 11) 1 r1 r3 6 r6 
False (0, 1, 5, 10, 12) 1 r1 r3 6 7 
False (0, 1, 5, 10, 13) 1 r1 r3 6 r7 
False (0, 1, 5, 10, 14) 1 r1 r3 6 8 
False (0, 1, 5, 10, 15) 1 r1 r3 6 r8 
False (0, 1, 5, 10, 16) 1 r1 r3 6 9 
False (0, 1, 5, 10, 17) 1 r1 r3 6 r9 
False (0, 1, 5, 10, 18) 1 r1 r3 6 發 
False (0, 1, 5, 10, 19) 1 r1 r3 6 中 
False (0, 1, 5, 11, 12) 1 r1 r3 r6 7 
False (0, 1, 5, 11, 13) 1 r1 r3 r6 r7 
False (0, 1, 5, 11, 14) 1 r1 r

False (0, 2, 2, 11, 16) 1 2 2 r6 9 
False (0, 2, 2, 11, 17) 1 2 2 r6 r9 
False (0, 2, 2, 11, 18) 1 2 2 r6 發 
False (0, 2, 2, 11, 19) 1 2 2 r6 中 
False (0, 2, 2, 12, 12) 1 2 2 7 7 
False (0, 2, 2, 12, 13) 1 2 2 7 r7 
False (0, 2, 2, 12, 14) 1 2 2 7 8 
False (0, 2, 2, 12, 15) 1 2 2 7 r8 
False (0, 2, 2, 12, 16) 1 2 2 7 9 
False (0, 2, 2, 12, 17) 1 2 2 7 r9 
False (0, 2, 2, 12, 18) 1 2 2 7 發 
False (0, 2, 2, 12, 19) 1 2 2 7 中 
False (0, 2, 2, 13, 14) 1 2 2 r7 8 
False (0, 2, 2, 13, 15) 1 2 2 r7 r8 
False (0, 2, 2, 13, 16) 1 2 2 r7 9 
False (0, 2, 2, 13, 17) 1 2 2 r7 r9 
False (0, 2, 2, 13, 18) 1 2 2 r7 發 
False (0, 2, 2, 13, 19) 1 2 2 r7 中 
False (0, 2, 2, 14, 14) 1 2 2 8 8 
False (0, 2, 2, 14, 15) 1 2 2 8 r8 
False (0, 2, 2, 14, 16) 1 2 2 8 9 
False (0, 2, 2, 14, 17) 1 2 2 8 r9 
False (0, 2, 2, 14, 18) 1 2 2 8 發 
False (0, 2, 2, 14, 19) 1 2 2 8 中 
False (0, 2, 2, 15, 16) 1 2 2 r8 9 
False (0, 2, 2, 15, 17) 1 2 2 r8 r9 
False (0, 2, 2, 15, 18) 1 2 2 r8 發 
False (0, 2, 2, 15, 19) 1 2 2 r8 

...

False (7, 11, 12, 17, 19) r4 r6 7 r9 中 
False (7, 11, 12, 18, 18) r4 r6 7 發 發 
False (7, 11, 12, 18, 19) r4 r6 7 發 中 
False (7, 11, 12, 19, 19) r4 r6 7 中 中 
False (7, 11, 13, 14, 14) r4 r6 r7 8 8 
False (7, 11, 13, 14, 15) r4 r6 r7 8 r8 
True (7, 11, 13, 14, 16) r4 r6 r7 8 9 
True (7, 11, 13, 14, 17) r4 r6 r7 8 r9 
False (7, 11, 13, 14, 18) r4 r6 r7 8 發 
False (7, 11, 13, 14, 19) r4 r6 r7 8 中 
True (7, 11, 13, 15, 16) r4 r6 r7 r8 9 
True (7, 11, 13, 15, 17) r4 r6 r7 r8 r9 
False (7, 11, 13, 15, 18) r4 r6 r7 r8 發 
False (7, 11, 13, 15, 19) r4 r6 r7 r8 中 
False (7, 11, 13, 16, 16) r4 r6 r7 9 9 
False (7, 11, 13, 16, 17) r4 r6 r7 9 r9 
False (7, 11, 13, 16, 18) r4 r6 r7 9 發 
False (7, 11, 13, 16, 19) r4 r6 r7 9 中 
False (7, 11, 13, 17, 18) r4 r6 r7 r9 發 
False (7, 11, 13, 17, 19) r4 r6 r7 r9 中 
False (7, 11, 13, 18, 18) r4 r6 r7 發 發 
False (7, 11, 13, 18, 19) r4 r6 r7 發 中 
False (7, 11, 13, 19, 19) r4 r6 r7 中 中 
True (7, 11, 14, 14, 14) r4 r6 8 8 8 
True (7, 11, 14, 14, 15) r4 r6 8 8 r8 


False (8, 9, 10, 13, 16) 5 r5 6 r7 9 
False (8, 9, 10, 13, 17) 5 r5 6 r7 r9 
False (8, 9, 10, 13, 18) 5 r5 6 r7 發 
False (8, 9, 10, 13, 19) 5 r5 6 r7 中 
False (8, 9, 10, 14, 14) 5 r5 6 8 8 
False (8, 9, 10, 14, 15) 5 r5 6 8 r8 
False (8, 9, 10, 14, 16) 5 r5 6 8 9 
False (8, 9, 10, 14, 17) 5 r5 6 8 r9 
False (8, 9, 10, 14, 18) 5 r5 6 8 發 
False (8, 9, 10, 14, 19) 5 r5 6 8 中 
False (8, 9, 10, 15, 16) 5 r5 6 r8 9 
False (8, 9, 10, 15, 17) 5 r5 6 r8 r9 
False (8, 9, 10, 15, 18) 5 r5 6 r8 發 
False (8, 9, 10, 15, 19) 5 r5 6 r8 中 
False (8, 9, 10, 16, 16) 5 r5 6 9 9 
False (8, 9, 10, 16, 17) 5 r5 6 9 r9 
False (8, 9, 10, 16, 18) 5 r5 6 9 發 
False (8, 9, 10, 16, 19) 5 r5 6 9 中 
False (8, 9, 10, 17, 18) 5 r5 6 r9 發 
False (8, 9, 10, 17, 19) 5 r5 6 r9 中 
False (8, 9, 10, 18, 18) 5 r5 6 發 發 
False (8, 9, 10, 18, 19) 5 r5 6 發 中 
False (8, 9, 10, 19, 19) 5 r5 6 中 中 
True (8, 9, 11, 12, 12) 5 r5 r6 7 7 
True (8, 9, 11, 12, 13) 5 r5 r6 7 r7 
True (8, 9, 11, 12, 14) 5 r5 r6 7 8 
True (8, 9, 11, 12, 15

False (8, 13, 16, 19, 19) 5 r7 9 中 中 
False (8, 13, 17, 18, 18) 5 r7 r9 發 發 
False (8, 13, 17, 18, 19) 5 r7 r9 發 中 
False (8, 13, 17, 19, 19) 5 r7 r9 中 中 
True (8, 13, 18, 18, 18) 5 r7 發 發 發 
False (8, 13, 18, 18, 19) 5 r7 發 發 中 
False (8, 13, 18, 19, 19) 5 r7 發 中 中 
True (8, 13, 19, 19, 19) 5 r7 中 中 中 
False (8, 14, 14, 14, 15) 5 8 8 8 r8 
False (8, 14, 14, 14, 16) 5 8 8 8 9 
False (8, 14, 14, 14, 17) 5 8 8 8 r9 
False (8, 14, 14, 14, 18) 5 8 8 8 發 
False (8, 14, 14, 14, 19) 5 8 8 8 中 
False (8, 14, 14, 15, 16) 5 8 8 r8 9 
False (8, 14, 14, 15, 17) 5 8 8 r8 r9 
False (8, 14, 14, 15, 18) 5 8 8 r8 發 
False (8, 14, 14, 15, 19) 5 8 8 r8 中 
False (8, 14, 14, 16, 16) 5 8 8 9 9 
False (8, 14, 14, 16, 17) 5 8 8 9 r9 
False (8, 14, 14, 16, 18) 5 8 8 9 發 
False (8, 14, 14, 16, 19) 5 8 8 9 中 
False (8, 14, 14, 17, 18) 5 8 8 r9 發 
False (8, 14, 14, 17, 19) 5 8 8 r9 中 
False (8, 14, 14, 18, 18) 5 8 8 發 發 
False (8, 14, 14, 18, 19) 5 8 8 發 中 
False (8, 14, 14, 19, 19) 5 8 8 中 中 
False (8, 14, 15, 1

False (9, 13, 17, 19, 19) r5 r7 r9 中 中 
True (9, 13, 18, 18, 18) r5 r7 發 發 發 
False (9, 13, 18, 18, 19) r5 r7 發 發 中 
False (9, 13, 18, 19, 19) r5 r7 發 中 中 
True (9, 13, 19, 19, 19) r5 r7 中 中 中 
False (9, 14, 14, 14, 15) r5 8 8 8 r8 
False (9, 14, 14, 14, 16) r5 8 8 8 9 
False (9, 14, 14, 14, 17) r5 8 8 8 r9 
False (9, 14, 14, 14, 18) r5 8 8 8 發 
False (9, 14, 14, 14, 19) r5 8 8 8 中 
False (9, 14, 14, 15, 16) r5 8 8 r8 9 
False (9, 14, 14, 15, 17) r5 8 8 r8 r9 
False (9, 14, 14, 15, 18) r5 8 8 r8 發 
False (9, 14, 14, 15, 19) r5 8 8 r8 中 
False (9, 14, 14, 16, 16) r5 8 8 9 9 
False (9, 14, 14, 16, 17) r5 8 8 9 r9 
False (9, 14, 14, 16, 18) r5 8 8 9 發 
False (9, 14, 14, 16, 19) r5 8 8 9 中 
False (9, 14, 14, 17, 18) r5 8 8 r9 發 
False (9, 14, 14, 17, 19) r5 8 8 r9 中 
False (9, 14, 14, 18, 18) r5 8 8 發 發 
False (9, 14, 14, 18, 19) r5 8 8 發 中 
False (9, 14, 14, 19, 19) r5 8 8 中 中 
False (9, 14, 15, 16, 16) r5 8 r8 9 9 
False (9, 14, 15, 16, 17) r5 8 r8 9 r9 
False (9, 14, 15, 16, 18) r5 8 r8

False (11, 12, 17, 19, 19) r6 7 r9 中 中 
True (11, 12, 18, 18, 18) r6 7 發 發 發 
False (11, 12, 18, 18, 19) r6 7 發 發 中 
False (11, 12, 18, 19, 19) r6 7 發 中 中 
True (11, 12, 19, 19, 19) r6 7 中 中 中 
True (11, 13, 14, 14, 14) r6 r7 8 8 8 
True (11, 13, 14, 14, 15) r6 r7 8 8 r8 
False (11, 13, 14, 14, 16) r6 r7 8 8 9 
True (11, 13, 14, 14, 17) r6 r7 8 8 r9 
False (11, 13, 14, 14, 18) r6 r7 8 8 發 
False (11, 13, 14, 14, 19) r6 r7 8 8 中 
True (11, 13, 14, 15, 16) r6 r7 8 r8 9 
True (11, 13, 14, 15, 17) r6 r7 8 r8 r9 
False (11, 13, 14, 15, 18) r6 r7 8 r8 發 
False (11, 13, 14, 15, 19) r6 r7 8 r8 中 
True (11, 13, 14, 16, 16) r6 r7 8 9 9 
True (11, 13, 14, 16, 17) r6 r7 8 9 r9 
False (11, 13, 14, 16, 18) r6 r7 8 9 發 
False (11, 13, 14, 16, 19) r6 r7 8 9 中 
False (11, 13, 14, 17, 18) r6 r7 8 r9 發 
False (11, 13, 14, 17, 19) r6 r7 8 r9 中 
True (11, 13, 14, 18, 18) r6 r7 8 發 發 
False (11, 13, 14, 18, 19) r6 r7 8 發 中 
True (11, 13, 14, 19, 19) r6 r7 8 中 中 
True (11, 13, 15, 16, 16) r6 r7 r8 9 9 
True 

In [12]:
# count the number of hands which can move to ready discard hand but not the best discard hand
count = 0
for i, hand in enumerate(valid_hands):
    if calc_point(hand) >= 5:
        continue
    hand_ready_list = [is_ready[valid_discard_hands.index(discard_hand)] for discard_hand in hand_to_discard_hands(hand)]
    if any(hand_ready_list):
        if all(hand_ready_list):
            continue
        #print(max_val, max_discard_hand)
        ready_max_val, ready_max_hand = max((discard_hand_value[valid_discard_hands.index(discard_hand)], discard_hand) for discard_hand in hand_to_discard_hands(hand) if is_ready[valid_discard_hands.index(discard_hand)])
        not_ready_max_val, not_ready_max_hand = max((discard_hand_value[valid_discard_hands.index(discard_hand)], discard_hand) for discard_hand in hand_to_discard_hands(hand) if not is_ready[valid_discard_hands.index(discard_hand)])

        if ready_max_val < not_ready_max_val:
            count += 1
#             print('hand', hand, hand2show_str(hand))
#             print('ready     {1} \t{2}\t{0}'.format(round(ready_max_val, 7), ready_max_hand, hand2show_str(ready_max_hand)))
#             print('not ready {1} \t{2}\t{0}\n'.format(round(not_ready_max_val, 7), not_ready_max_hand, hand2show_str(not_ready_max_hand)))
print(count)

2054


In [22]:
for dora in range(20):
    valid_hands = generate_all_valid_hands(MAX_LEN_HAND, dora)
    hand_to_optimal_tiles = np.load(
                './hand2optimal/hand2optimal_{}_{}.npy'.format('09', dora)).tolist()
    #print(len(valid_hands))
    #print(len(hand_to_optimal_tiles))
    opt_dict = {}
    for i in range(len(valid_hands)):
        hand = valid_hands[i]
        opt_tiles = hand_to_optimal_tiles[i]
        if opt_tiles == None:
            continue
        else:
            opt_dict[hand] = opt_tiles
    with open('./hand2optimal/hand2optdict_{}_{}.pickle'.format('09', dora), 'wb') as f:
        pickle.dump(opt_dict, f)

In [23]:
# with open('./hand2optimal/hand2optdict_09_0.pickle', 'rb') as f:
#     d = pickle.load(f)